In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from pimpa.trade_models.portfolio import Portfolio
from pimpa.evaluators.ccr_valuation_session import CCR_Valuation_Session
from data.configuration.global_parameters import global_parameters
from pimpa.utils.calendar_utils import transform_dates_to_time_differences

from pimpa.pricing_models.interest_rate_swap_pricer import InterestRateSwapPricer
from pimpa.pricing_models.equity_european_option_pricer import EquityEuropeanOptionPricer
from pimpa.trade_models.interest_rate_swap import InterestRateSwap
from pimpa.trade_models.equity_european_option import EquityEuropeanOption
from pimpa.market_data_objects.market_data_builder import MarketDataBuilder
from pimpa.utils.calendar_utils import generate_simulation_dates_schedule

from data.configuration.global_parameters import global_parameters, calibration_parameters


# Construction of a portfolio based on sourced data

1) Create a portfolio object
2) Check the portfolio object attributes & methods 
3) Check the data that has been sourced (master_ledger, desks, counterparty)

In [2]:
netting_agreement_id = 23
test_portfolio_1 = Portfolio(netting_agreement_id)
print('-----------------------------')
print('BEFORE LOAD:')
print(test_portfolio_1)
print('-----------------------------')
test_portfolio_1.load(global_parameters)
print('AFTER LOAD:')
print(test_portfolio_1)
print('-----------------------------')
print(test_portfolio_1.netting_sets)
print('-----------------------------')

-----------------------------
BEFORE LOAD:
Portfolio with netting_agreement_id: 23
NOT AVAILABLE
-----------------------------
AFTER LOAD:
Portfolio with netting_agreement_id: 23
- including trade_IDs: [1234, 1235, 1236]
- including netting_sets: ['23_MAIN', '23_SINGLETON_1']
- with underlyings: {'USD_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE'}
- with 65 valuation points.
AVAILABLE
-----------------------------
{'23_MAIN': [1234, 1235], '23_SINGLETON_1': [1236]}
-----------------------------


1) Check portfolio trades
2) Check trade object and methods

In [3]:
print(test_portfolio_1.trade_inventory)
print('-----------------------------')
for _, trade in test_portfolio_1.trade_inventory.items():
    print(trade)
    print('-----------------------------')

{1234: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x7fa1a9669ca0>, 1235: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x7fa1a9669e80>, 1236: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x7fa1d8739190>}
-----------------------------
Trade with trade_id: 1234
- trade type: IRS
- asset class: IR
- trade currency: USD
- trade underlyings: ['USD_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE']
- trade attributes keys: ['notional', 'currency', 'floating_rate', 'K', 'payer/receiver', 'first_fixing_date', 'last_fixing_date', 'first_payment_date', 'last_payment_date', 'payments_frequency', 'maturity', 'fixings_schedule', 'payments_schedule'])
- with 41 valuation points.
AVAILABLE
-----------------------------
Trade with trade_id: 1235
- trade type: IRS
- asset class: IR
- trade currency: USD
- trade underlyings: ['USD_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE']
- trade attributes keys: ['notional', 'currency', 'floating_rate', 'K', 'payer/r

1) Check collateral agreements

In [4]:
print('-----------------------------')
print(test_portfolio_1.vm_collateral_agreements)
print('-----------------------------')

-----------------------------
{'VM_1': {'trade_ids': [1234, 1235], 'contractual_terms':   WE_POST_P WE_RECEIVE_R  T_P  T_R  MTA_P  MTA_R
0       YES          YES    2  1.5    0.5    0.3}}
-----------------------------


# Trade valuation (IRS)

1) Construct a IRS trade with child class InterestRateSwap from trade_id
2) Load the trade

In [5]:
trade_id = 1234
risk_factors = pd.read_csv(global_parameters['prototype_data_paths']['RFs_attributes'] + global_parameters['prototype_data_files']['RFs_attributes']['all_RFs_mapping'])
trade = InterestRateSwap(trade_id)
trade.load(global_parameters, risk_factors)
print('-----------------------------')
print(trade)
print('-----------------------------')

-----------------------------
Trade with trade_id: 1234
- trade type: IRS
- asset class: IR
- trade currency: USD
- trade underlyings: ['USD_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE']
- trade attributes keys: ['notional', 'currency', 'floating_rate', 'K', 'payer/receiver', 'first_fixing_date', 'last_fixing_date', 'first_payment_date', 'last_payment_date', 'payments_frequency', 'maturity', 'fixings_schedule', 'payments_schedule'])
- with 41 valuation points.
AVAILABLE
-----------------------------


1) Construct an IRS pricer with child class InterestRateSwapPricer
2) Create the risk_factor_objects with class MarketDataBuilder (Notice that we do not calibrate the RFE models but we still need to know which are simulated and which are not)
3) Identify the dependencies for the pricer

In [6]:
pricer = InterestRateSwapPricer('IRS_Pricer')
risk_factor_objects = MarketDataBuilder().get_risk_factors(trade.trade_underlyings, global_parameters)
print('-----------------------------')
print(risk_factor_objects)
print('-----------------------------')
pricer_dependencies = pricer.get_market_dependencies(trade.trade_underlyings, risk_factor_objects,calibration_parameters)
print(pricer_dependencies)
print('-----------------------------')

-----------------------------
{'USD_ZERO_YIELD_CURVE': RiskFactor(name='USD_ZERO_YIELD_CURVE', asset_class='IR', asset_type='DISCOUNT_CURVE', currency='USD', simulated=True, model=<pimpa.scenario_generation.hw1f.HW1F object at 0x7fb6084a0550>, model_name='HW1F', reference=None), 'USD_LIBOR_3M_CURVE': RiskFactor(name='USD_LIBOR_3M_CURVE', asset_class='IR', asset_type='SPREAD_TO_DISCOUNT_CURVE', currency='USD', simulated=False, model=None, model_name='NOT_AVAILABLE', reference='USD_ZERO_YIELD_CURVE')}
-----------------------------
{('Pricing_HW1F_calibration', 'USD_ZERO_YIELD_CURVE'), ('spread_to_discount_curve', 'USD_LIBOR_3M_CURVE'), ('historical_fixings', 'USD_ZERO_YIELD_CURVE'), ('historical_fixings', 'USD_LIBOR_3M_CURVE')}
-----------------------------


1) Source market data based on pricer dependencies and using the folders/files pointers in global_parameters (notice that we skipped RFE models dependencies since we do not use them here)
2) Calibrate the pricer 

In [7]:
market_data = MarketDataBuilder().load_market_data(pricer_dependencies, global_parameters)
print('-----------------------------')
print(market_data)
print('-----------------------------')
pricer.calibrate(market_data, calibration_parameters)
print(pricer)
print('-----------------------------')

-----------------------------
{'Pricing_HW1F_calibration': {'USD_ZERO_YIELD_CURVE': {'alpha': 0.05, 'volatility': 0.01, 'rate_curve': <pimpa.market_data_objects.curve.Curve object at 0x7fb61260af10>}}, 'spread_to_discount_curve': {'USD_LIBOR_3M_CURVE': <pimpa.market_data_objects.curve.Curve object at 0x7fb6084a0310>}, 'historical_fixings': {'USD_ZERO_YIELD_CURVE': 2020-01-01    0.01
2019-12-31    0.02
2019-12-30    0.02
2019-12-29    0.02
2019-12-28    0.02
              ... 
2018-12-18    0.02
2018-12-17    0.02
2018-12-16    0.02
2018-12-15    0.02
2018-12-14    0.02
Name: USD_ZERO_YIELD_CURVE, Length: 384, dtype: float64, 'USD_LIBOR_3M_CURVE': 2020-01-01    0.002
2019-12-31    0.002
2019-12-30    0.002
2019-12-29    0.002
2019-12-28    0.002
              ...  
2018-12-18    0.002
2018-12-17    0.002
2018-12-16    0.002
2018-12-15    0.002
2018-12-14    0.002
Name: USD_LIBOR_3M_CURVE, Length: 384, dtype: float64}}
-----------------------------
-----------------------------


1) Generate a fictious valuation grid
2) Generate fictious scenarios for the only simulated RF ('USD_ZERO_YIELD_CURVE')
3) Use the IRS pricer object to value the fictious scenarios 

In [8]:
starting_date = '2020-01-01'
final_date = '2040-01-01'
valuation_frequency = 'monthly'
nr_paths = 1000
global_parameters['n_paths'] = nr_paths
valuation_dates = generate_simulation_dates_schedule(starting_date, final_date, valuation_frequency, global_parameters)
scenarios = {'USD_ZERO_YIELD_CURVE': np.random.normal(loc=0.01, scale=0.005, size=(nr_paths, len(valuation_dates)))}

trade_mtms = pricer.price_single_trade(trade, valuation_dates, scenarios, market_data, global_parameters)

# Trade valuation (EUR EQ OPT)

1) Construct a EUR EQ OPT trade with child class EquityEuropeanOption from trade_id
2) Load the trade

In [9]:
trade_id = 2345
risk_factors = pd.read_csv(global_parameters['prototype_data_paths']['RFs_attributes'] + global_parameters['prototype_data_files']['RFs_attributes']['all_RFs_mapping'])
trade = EquityEuropeanOption(trade_id)
trade.load(global_parameters, risk_factors)

print(trade)

Trade with trade_id: 2345
- trade type: EQ_EUR_OPT
- asset class: EQ
- trade currency: USD
- trade underlyings: ['CREDIT_SUISSE_SHARE', 'USD_ZERO_YIELD_CURVE', 'CREDIT_SUISSE_IMPLIED_VOLATILITY_SURFACE']
- trade attributes keys: ['notional', 'currency', 'underlying', 'K', 'put/call', 'long/short', 'maturity'])
- with 0 valuation points.
AVAILABLE


1) Construct an EQ_EUR_OPT pricer with child class EquityEuropeanOptionPricer
2) Create the risk_factor_objects with class MarketDataBuilder (Notice that we do not calibrate the RFE models but we still need to know which are simulated and which are not)
3) Identify the dependencies for the pricer

In [10]:
pricer = EquityEuropeanOptionPricer('EQ_EUR_OPT_Pricer')
risk_factor_objects = MarketDataBuilder().get_risk_factors(trade.trade_underlyings, global_parameters)
pricer_dependencies = pricer.get_market_dependencies(trade.trade_underlyings, risk_factor_objects,calibration_parameters)
print(pricer_dependencies)

{('Pricing_HW1F_calibration', 'USD_ZERO_YIELD_CURVE'), ('historical_fixings', 'USD_ZERO_YIELD_CURVE'), ('equity_implied_volatility_surface', 'CREDIT_SUISSE_IMPLIED_VOLATILITY_SURFACE')}


1) Source market data based on pricer dependencies and using the folders/files pointers in global_parameters (notice that we skipped RFE models dependencies since we do not use them here)
2) Calibrate the pricer 

In [11]:
market_data = MarketDataBuilder().load_market_data(pricer_dependencies, global_parameters)
pricer.calibrate(market_data, calibration_parameters)

print(pricer)

1) Generate a fictious valuation grid
2) Generate fictious scenarios for the simulated RFs ('USD_ZERO_YIELD_CURVE' and 'CREDIT_SUISSE_SHARE')
3) Use the EQ_EUR_OPT pricer object to value the fictious scenarios 

In [12]:
starting_date = '2017-01-01'
final_date = '2022-01-01'
valuation_frequency = 'monthly'
nr_paths = 1000
global_parameters['n_paths'] = nr_paths
valuation_dates = generate_simulation_dates_schedule(starting_date, final_date, valuation_frequency, global_parameters)
scenarios = {'USD_ZERO_YIELD_CURVE': np.random.normal(loc=0.01, scale=0.005, size=(nr_paths, len(valuation_dates))), \
            'CREDIT_SUISSE_SHARE':10*np.random.lognormal(mean=0.0, sigma=0.4, size=(nr_paths, len(valuation_dates)))}
trade_mtms = pricer.price_single_trade(trade, valuation_dates, scenarios, market_data, global_parameters)
print('-----------------------------')
print(scenarios)
print('-----------------------------')
print(trade_mtms)
print('-----------------------------')

-----------------------------
{'USD_ZERO_YIELD_CURVE': array([[0.01472774, 0.0073373 , 0.01081822, ..., 0.01147231, 0.00859376,
        0.01015637],
       [0.00932633, 0.00995652, 0.00617551, ..., 0.005065  , 0.00179165,
        0.00449957],
       [0.01569924, 0.01753808, 0.01647014, ..., 0.00764985, 0.0068205 ,
        0.01649578],
       ...,
       [0.01118131, 0.00160661, 0.00461421, ..., 0.00665567, 0.00557066,
        0.00692752],
       [0.01078097, 0.00976527, 0.00783458, ..., 0.00623163, 0.00771967,
        0.00353352],
       [0.00419302, 0.01136222, 0.00872512, ..., 0.00113122, 0.00959767,
        0.00793998]]), 'CREDIT_SUISSE_SHARE': array([[24.93314507, 11.16434884, 11.71651252, ...,  7.82638628,
        11.80762493, 15.85938225],
       [12.87602661,  8.39039552, 13.051736  , ..., 13.01644803,
        18.40240698,  9.33036047],
       [28.02633286,  6.05766272,  8.50370805, ..., 14.9591227 ,
         6.53176858,  5.9431219 ],
       ...,
       [ 5.44828374,  9.33100031